# Не забыть приделать сюда комменты!!!

## Импорты

In [173]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re
import pandas as pd

## Краулер
качаем [рассказы Булгакова](http://az.lib.ru/b/bulgakow_m_a/) из раздела "записки юного врача"

In [3]:
link_list = ["http://az.lib.ru/b/bulgakow_m_a/text_01_1926_polotentze_s_petuhom.shtml",
"http://az.lib.ru/b/bulgakow_m_a/text_02_1926_viyuga.shtml",
"http://az.lib.ru/b/bulgakow_m_a/text_03_1925_stalnoe_gorlo.shtml",
"http://az.lib.ru/b/bulgakow_m_a/text_04_1925_tma_egipetskaya.shtml",
"http://az.lib.ru/b/bulgakow_m_a/text_05_1925_kreshenie_povorotom.shtml",
"http://az.lib.ru/b/bulgakow_m_a/text_06_1925_propavshiy_glaz.shtml",
"http://az.lib.ru/b/bulgakow_m_a/text_07_1925_zvezdnaya_syp.shtml",
"http://az.lib.ru/b/bulgakow_m_a/text_08_1925_ya_ubil.shtml",
"http://az.lib.ru/b/bulgakow_m_a/text_09_1927_morfiy.shtml"]

In [4]:
user_agent = UserAgent()

def follow_me(link) -> str:
    """Функкия для перехода по ссылкам, притворяясь случайным браузером"""
    response = requests.get(link, headers={'User-Agent':user_agent.random})
    return response.text

In [109]:
def work_on_story(link, i):
    """Функция, сохраняющая в файл текст, чистящая его от лишних пробельных символов. 
    Первой стокой сохраняет ссылку на текст, второй - название текста и имя автора"""
    soup = BeautifulSoup(follow_me(link), 'html.parser')
    name1 = soup.find("h4")
    name = name1.get_text().rstrip()
    name = name.replace("\n", "")
    texts = name1.find_next_siblings()
    # Убираем заголовки - номера глав, которые лежали на сайте как попало
    body_text = " ".join([text.get_text() for text in texts if re.fullmatch(r'\s*I+\s*', text.get_text()) is None])
    annotation = body_text.find("------------------------------------------------------------")
    # Убираем аннотацию (о месте первой публикации рассказа)
    if annotation != -1:
        body_text = body_text[:annotation]
    body_text = body_text.replace("--", "—")
    body_text = body_text.replace("", "-")
    body_text = body_text.replace('\n', " ")
    body_text = body_text.replace(" ", " ")
    # если в процессе получилось несколько пробелов подряд, убираем
    while body_text.count("  ") != 0:
        body_text = body_text.replace("  ", " ")
    with open(f"texts/base{i}.txt", "a+", encoding="UTF-8") as r:
        print(link, file=r)
        print(name + ".", file=r)
        print(body_text, file=r)
        # На всякий случай заменяем все переносы строки на пробел 

In [89]:
for i in range(0, len(link_list)):
    work_on_story(link_list[i], i+1)
# собственно, проходимся по всем рассказам и получаем файлы с текстами

## Разбиение на предложения

In [175]:
def parsing(text) -> list:
    """С помощью регулярных выражений находит границы предложений
    и делит по ним текст. Прямая речь плохо делится, 
    поэтому её мы соединяем обратно. Возвращает список строк"""
    text = text.replace("* * * ", "")
    # замена ещё одной лишней пунктуации 
    sentences = re.split(r'(?<=[.?!])(?=\s+)', text.rstrip())
    to_pop = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        if i > 0:
            if sentence[1] == "—":
                answer = sentence[3].islower()
            else:
                answer = sentence[1:2].islower()
        else:
            answer = False
        # если получается так, что предложение начинается с 
        # маленькой буквы, его необходимо присоединить к предыдущему 
        # (здесь предложения в начале ещё имеют пробел, поэтому такие индексы) 
        if answer is True:
            sentences[i-1] += sentence
            to_pop.append(f"{i}")
        sentence = sentence.lstrip()
    to_pop.reverse()  # чтобы при удалении элементов не поехали индексы
    for u in to_pop:
        sentences.pop(int(u))
    return sentences

In [176]:
# проходимся по всем файлам с текстами, делим их на предложения
# и создаём список из списков "предложение - источник - ссылка",
# чтобы сделать датафрейм и сохранить его в csv
to_df = []
for n in range(1, 10):
    with open(f"texts/base{n}.txt", "r", encoding="UTF-8") as t:
        link, name, text = t.readlines()
    sentences = parsing(text)
    s_to_df = [[sentence, name.rstrip(), link.rstrip()] for sentence in sentences]
    to_df.extend(s_to_df)
df = pd.DataFrame(to_df, columns=["sentence", "name", "link"])
df.to_csv('all_sentences.csv', index=False, sep="\t")